<a href="https://colab.research.google.com/github/SaqlainHussainShah/GenAI/blob/main/finetune_gemma_medal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://storage.googleapis.com/gweb-uniblog-publish-prod/images/gemma-header.width-1200.format-webp.webp" width="100%">

## Instruct Fine-tuning [Gemma](https://blog.google/technology/developers/gemma-open-models/) using qLora and Supervise Finetuning

This is a comprahensive notebook and tutorial on how to fine tune the `gemma-7b-it` Model

All the code will be available on my Github. Do drop by and give a follow and a star.
[adithya-s-k](https://github.com/adithya-s-k)
\
[Github Code](https://github.com/adithya-s-k/LLM-Cookbook/blob/main/LLMs/Gemma/finetune-gemma.ipynb)

I also post content about LLMs and what I have been working on Twitter.
[AdithyaSK (@adithya_s_k) / X](https://twitter.com/adithya_s_k)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prerequisites

Before delving into the fine-tuning process, ensure that you have the following prerequisites in place:

1. **GPU**: [gemma-2b](https://huggingface.co/google/gemma-2b) - can be finetuned on T4(free google colab) while [gemma-7b](https://huggingface.co/google/gemma-7b) requires an A100 GPU.
2. **Python Packages**: Ensure that you have the necessary Python packages installed. You can use the following commands to install them:

Let's begin by checking if your GPU is correctly detected:

In [9]:
!nvidia-smi

Mon Feb 26 07:23:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [10]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Now we specify the model ID and then we load it with our previously defined quantization configuration.Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [12]:
# if you are using google colab

# import os
# from google.colab import userdata
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [15]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [16]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  
model


  ```python
def fibonacci(n):
    """
    Fibonacci sequence generator.

    Args:
        n: The index of the sequence.

    Returns:
        The nth number in the Fibonacci sequence.
    """

    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)


# Print the first 10 numbers in the Fibonacci sequence.
for i in range(10):
    print(fibonacci(i))
```


## Step 3 - Load dataset for finetuning

### Lets Load the Dataset

For this tutorial, we will fine-tune Mistral 7B Instruct for code generation.

We will be using this [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) which is curated by [TokenBender (e/xperiments)](https://twitter.com/4evaBehindSOTA) and is an excellent data source for fine-tuning models for code generation. It follows the alpaca style of instructions, which is an excellent starting point for this task. The dataset structure should resemble the following:

```json
{
  "instruction": "Create a function to calculate the sum of a sequence of integers.",
  "input": "[1, 2, 3, 4, 5]",
  "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}
```

In [ ]:
from datasets import load_dataset

dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['output', 'input', 'text', 'instruction'],
    num_rows: 121959
})

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
df = dataset.to_pandas()
df.head(10)

,output,input,text,instruction
0,# Python code\ndef sum_sequence(sequence):\n ...,"[1, 2, 3, 4, 5]",Below is an instruction that describes a task....,Create a function to calculate the sum of a se...
1,"def add_strings(str1, str2):\n """"""This func...","str1 = ""Hello ""\nstr2 = ""world""",Below is an instruction that describes a task....,Develop a function that will add two strings
2,#include <map>\n#include <string>\n\nclass Gro...,,Below is an instruction that describes a task....,Design a data structure in C++ to store inform...
3,def bubble_sort(arr):\n n = len(arr)\n \n ...,"[3, 1, 4, 5, 9, 0]",Below is an instruction that describes a task....,Implement a sorting algorithm to sort a given ...
4,import UIKit\n\nclass ExpenseViewController: U...,Not applicable,Below is an instruction that describes a task....,Design a Swift application for tracking expens...
5,<?php\n$timestamp = $_GET['timestamp'];\n\nif(...,Not Applicable,Below is an instruction that describes a task....,Create a REST API to convert a UNIX timestamp ...
6,import requests\nimport re\n\ndef crawl_websit...,website: www.example.com \ndata to crawl: phon...,Below is an instruction that describes a task....,Generate a Python code for crawling a website ...
7,"[x*x for x in [1, 2, 3, 5, 8, 13]]",,Below is an instruction that describes a task....,Create a Python list comprehension to get the ...
8,SELECT * FROM products ORDER BY price DESC LIM...,,Below is an instruction that describes a task....,Create a MySQL query to find the most expensiv...
9,public class Library {\n \n // map of books in...,Not applicable,Below is an instruction that describes a task....,Create a data structure in Java for storing an...


Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

### Formatting the Dataset

Now, let's format the dataset in the required [gemma instruction formate](https://huggingface.co/google/gemma-7b-it).

> Many tutorials and blogs skip over this part, but I feel this is a really important step.

```
<start_of_turn>user What is your favorite condiment? <end_of_turn>
<start_of_turn>model Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavor to whatever I'm cooking up in the kitchen!<end_of_turn>
```

You can use the following code to process your dataset and create a JSONL file in the correct format:

In [17]:
# def generate_prompt(data_point):
#     """Gen. input text based on a prompt, task instruction, (context info.), and answer

#     :param data_point: dict: Data point
#     :return: dict: tokenzed prompt
#     """
#     prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
#                'appropriately completes the request.\n\n'
#     # Samples with additional context into.
#     if data_point['input']:
#         text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
#     # Without
#     else:
#         text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
#     return text

# # add the "prompt" column in the dataset
# text_column = [generate_prompt(data_point) for data_point in dataset]
# dataset = dataset.add_column("prompt", text_column)

NameError: name 'dataset' is not defined

We'll need to tokenize our data so the model can understand.


In [ ]:
# dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
# dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/121959 [00:00<?, ? examples/s]

Split dataset into 90% for training and 10% for testing

In [ ]:
# dataset = dataset.train_test_split(test_size=0.2)
# train_data = dataset["train"]
# test_data = dataset["test"]

### After Formatting, We should get something like this

```json
{
"text":"<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>",
"instruction":"Create a function to calculate the sum of a sequence of integers",
"input":"[1, 2, 3, 4, 5]",
"output":"# Python code def sum_sequence(sequence): sum = 0 for num in,
 sequence: sum += num return sum",
"prompt":"<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>"

}
```

While using SFT (**[Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/main/en/sft_trainer)**) for fine-tuning, we will be only passing in the “text” column of the dataset for fine-tuning.

In [19]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/medal_dataset_finetuning/small_test_data.csv')

In [20]:
df.columns

Index(['Unnamed: 0', 'ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'], dtype='object')

In [21]:
df.head()

,Unnamed: 0,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,0,2069316,we developed an animal model of chronic allerg...,89,functional residual capacity
1,1,6967959,pyogenic granulomas represent the aquisition o...,34,pyogenic granuloma
2,2,3492721,the l immunotype lipopolysaccharide lps of nei...,97,phosphorylethanolamine
3,3,8388738,inotropic reserve identified by dobutamine or ...,47,stress echocardiography
4,4,6234238,pyridoxinedependent seizure is a rare autosoma...,57,severe mental retardation


In [34]:
df.rename(columns = {'TEXT':'instruction'}, inplace = True)
df.rename(columns = {'LABEL': 'output'}, inplace=True)
df['input'] = df['instruction']
df['text'] = 'Below is a passage that describes a medical terminology or disease. Classify the text topic'

In [35]:
# import pandas as pd
# data = {
#     "output": output_list,
#     "input": input_list,
#     "text": text_list,
#     "instruction": instruction_list
# }

# # Create a DataFrame from the dictionary
# df = pd.DataFrame(data)

In [36]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)


In [37]:
type(dataset)

datasets.arrow_dataset.Dataset

In [38]:
df.head()

,Unnamed: 0,ABSTRACT_ID,instruction,LOCATION,output,input,text
0,0,2069316,we developed an animal model of chronic allerg...,89,functional residual capacity,we developed an animal model of chronic allerg...,Below is a passage that describes a medical te...
1,1,6967959,pyogenic granulomas represent the aquisition o...,34,pyogenic granuloma,pyogenic granulomas represent the aquisition o...,Below is a passage that describes a medical te...
2,2,3492721,the l immunotype lipopolysaccharide lps of nei...,97,phosphorylethanolamine,the l immunotype lipopolysaccharide lps of nei...,Below is a passage that describes a medical te...
3,3,8388738,inotropic reserve identified by dobutamine or ...,47,stress echocardiography,inotropic reserve identified by dobutamine or ...,Below is a passage that describes a medical te...
4,4,6234238,pyridoxinedependent seizure is a rare autosoma...,57,severe mental retardation,pyridoxinedependent seizure is a rare autosoma...,Below is a passage that describes a medical te...


In [39]:
df['output'].head()

0    functional residual capacity
1              pyogenic granuloma
2          phosphorylethanolamine
3         stress echocardiography
4       severe mental retardation
Name: output, dtype: object

In [40]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is a text passage that describes a medical topic. Write a response that ' \
               'appropriately classify the disease or topic in the passage .\n\n'
    # Samples with additional context into.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]}  <end_of_turn>\n<start_of_turn>model Above text talks about {data_point["output"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model Above text talks about {data_point["output"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [41]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [42]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [43]:
print(test_data)

Dataset({
    features: ['Unnamed: 0', 'ABSTRACT_ID', 'instruction', 'LOCATION', 'output', 'input', 'text', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 20000
})


## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [44]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [45]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
 

In [46]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [47]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'k_proj', 'q_proj', 'v_proj', 'up_proj', 'gate_proj', 'o_proj']


In [48]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [49]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


## Step 5 - Run the training!

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
# import transformers

# tokenizer.pad_token = tokenizer.eos_token


# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs_mistral_b_finance_finetuned_test",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [50]:
from huggingface_hub import notebook_login
notebook_login()

In [51]:
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="saqlainshah",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        push_to_hub=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


## Lets start training

In [53]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,5.690500
2,5.547800
3,5.586700
4,4.631800
5,4.423400
6,4.571600
7,4.265600
8,4.010000
9,3.448900
10,3.775900


TrainOutput(global_step=100, training_loss=3.182814846038818, metrics={'train_runtime': 789.93, 'train_samples_per_second': 0.506, 'train_steps_per_second': 0.127, 'total_flos': 1190162568757248.0, 'train_loss': 3.182814846038818, 'epoch': 0.01})

 Share adapters on the 🤗 Hub

In [52]:
new_model = "gemma_2b_finetuned_medal" #Name of the model you will be pushing to huggingface model hub

In [54]:
trainer.model.save_pretrained(new_model)

In [55]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/saqlainshah/gemma_2b_finetuned_medal/commit/3ad5949a51211f40fc55af37ee9d56f3e1b283b0', commit_message='Upload tokenizer', commit_description='', oid='3ad5949a51211f40fc55af37ee9d56f3e1b283b0', pr_url=None, pr_revision=None, pr_num=None)

## Test out Finetuned Model

In [57]:
result = get_completion(query="we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in c in bal total cells were x ml in a in b and in c macrophages in bal were in a in b and in c neutrophils were in a in b in c eosinophils were in a in b and in c p less than group c versus group a total proteins albumin ALP phosphatase and fibronectin did not differ between groupsabstract truncated at words", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in c in bal total cells were x ml in a in b and in c macrophages in bal were in a in b and in c neutrophils were in a in b in c eosinophils were in a in 